In [14]:
import time 
import math
import datetime
import os
import json
import hashlib
import functools
import traceback
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
import undetected_chromedriver as uc
from undetected_chromedriver import ChromeOptions
from loguru import logger
import pandas as pd

import xpath as xp
from settings import *

In [ ]:
def error_handler(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        try:
            if func.__name__ == 'get_family':
                d = self.d1
                try:
                    result = func(self, *args, **kwargs)
            elif func.__name__ == 'get_categories':
                d = self.d2
            else:
                d = self.d3

            url = d.current_url
            result = func(self, *args, **kwargs)
        except Exception as e:
            traceback.print_exc()
            if isinstance(e, NoSuchElementException):
                logger.error(e)
            else:
                logger.error(e)
                self.driver.quit()
                self.init_driver(url)
                result = func(self, *args, **kwargs)
        return result
    return wrapper

In [15]:
class CacheHTMLPages:
    def __init__(self):
        self.create_cache_folder_with_date()
        
    def create_cache_folder_with_date(self):
        self.cache_path = f'{os.getcwd()}/cache/{datetime.datetime.now().strftime("%Y-%m-%d")}'
        if not os.path.exists(self.cache_path):
            os.makedirs(self.cache_path)

    def set_cache(self, driver):
        url = driver.current_url
        try:
            html = driver.find_element("xpath", "//*").get_attribute("outerHTML")
        except Exception as e:
            logger.error(e)
            html = None
        url_hash = hashlib.sha1(str.encode(url)).hexdigest()
        url_path = f'{self.cache_path}/{url_hash}'
        if not os.path.exists(url_path):
            os.makedirs(url_path)
        
        metadata = {
            "url": url,
            "hashed_url": url_hash + '.html',
            "cache_path": self.cache_path,
            "size": f"{math.ceil(len(html) / 1024)} KB",
            "created_at": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            
        }

        with open(f'{url_path}/{metadata["hashed_url"]}', 'w') as f:
            f.write(html.replace('href="/', 'href="https://www.mcmaster.com/'))
        with open(f'{url_path}/metadata.json', 'a') as f:
            f.write(json.dumps(metadata) + '\n')

    def get_cache(self, driver, url):
        if 'http' not in url:
            url = f'{BASE_URL}{url}'
        url_hash = hashlib.sha1(str.encode(url)).hexdigest()
        url_path = f'{self.cache_path}/{url_hash}'
        if os.path.exists(url_path):
            driver.get(f'file://{url_path}/{url_hash}.html')
            return True
        else:
            logger.info(f'Caching => {url}')
            return False


In [16]:
class Driver(CacheHTMLPages):
    def __init__(self, args=ARGUMENTS) -> None:
        super().__init__()
        options = ChromeOptions()
        for i in args:
            options.add_argument(i)
        self.driver = uc.Chrome(headless=False, use_subprocess=True, options=options)
    def get(self, url):
        # get cache file if exists
        if not self.get_cache(self.driver, url):
            self.driver.get(url)
            time.sleep(3)
            self.set_cache(self.driver)

In [17]:
class ExtractData:
    def __init__(self, body, head, path, filename):
        self.headers = dict()
        self.body_data = list()   
        self.process_table_headers(head)
        self.process_table_body(body) 
        self.merge_headers()
        self.save_data()
    
    def validate_headers(self):
        if 'title-0' not in self.headers:
            raise Exception('Headers Doesnt contains title-0')
        if not len(self.headers) * len(self.headers['title-0']) == sum([len (v) for _ , v in self.headers.items()]):
            raise Exception('Validation Failed')

    def merge_headers(self):
        headers_len = len(self.headers['title-0'])
        avg_body_len = math.ceil(sum([len(i) for i in self.body_data]) / len(self.body_data))

        if self.headers == avg_body_len:
            pass
        # add one more column in the headers as
        # body has a row_header
        elif headers_len < avg_body_len:
            diff = avg_body_len - headers_len
            print(f'Row headers found: {diff}')
            for k, v in self.headers.items():
                v.append('Row Header')
                self.headers[k] = v
    
    def process_table_headers(self, head):
        table_titles = head.find_elements(By.XPATH, './tr')
        for index, tr in enumerate(table_titles):
            # check if <td> is present i.e title-0
            if tr.find_elements(By.XPATH, './td'):
                title0 = [i.text for i in tr.find_elements(By.XPATH, './td')]
                self.headers[f'title-{len(table_titles) - 1 - index}'] = title0
            # rest titles i.e. title-(n)
            else:
                titlen = []
                ths = tr.find_elements(By.XPATH, './th')
                for tr1 in ths:
                    col_span = tr1.get_attribute('colspan')
                    titlen += ([tr1.text] * int(col_span))
                
                self.headers[f'title-{len(table_titles) - 1 - index}'] = titlen
        # validate headers
        self.validate_headers()

    def process_table_body(self, body):
        table_body = body.find_elements(By.XPATH, './tr')
        row_headers = ''
        for index, tr in enumerate(table_body):
            # row is a header actually
            if tr.find_elements(By.XPATH, './th'):
                row_headers = tr.text
                continue
            else:
                tds = tr.find_elements(By.XPATH, './td')
                row = []
                for index1, td in enumerate(tds):
                    row.append(td.text)
                
                if row_headers:
                    row.append(row_headers)

            self.body_data.append(row)
    def save_data(self, file_name):
        df = pd.DataFrame(self.body_data, columns=[v for k, v in self.headers.items()])



        

In [18]:
class MCMaster(Driver):
    def __init__(self):
        self.d1 = Driver()
        self.d2 = Driver()
        self.d3 = Driver()

    def get_family(self, family_url, family_name):
        self.d1.get(family_url)
        categories = self.d1.driver.find_elements(By.XPATH, xp.CATEGORIES)
        for i in categories:
            print(i)
            category_url, category_name = i.get_attribute('href'), i.text.replace(', ', '-').replace(' & ', '-').replace(' ', '-')
            print(category_name, category_url)
            self.get_categories({'family_url': family_url, 'family_name': family_name, 'path': family_name.lower().relace(' ', '-')}, category_url, category_name)
    

    def get_categories(self, meta, category_url, category_name):
        self.d2.get(category_url)
        products = self.d2.driver.find_elements(By.XPATH, xp.PRODUCTS)
        for i in products:
            product_urls = [j.get_attribute('href') for j in i.find_elements(By.XPATH, xp.PRODUCT_URLS)]
            for product_url in product_urls:
                meta['category_url'], meta['category_name'], meta['path'] = category_url, category_name, meta['path'] + '/' + category_name.lower().replace(' ', '-')
                self.get_products(meta, product_url)
                break


    def get_products(self, meta, product_url):
        print(meta, product_url)
        meta['product_url'] = product_url
        self.d3.get(product_url)
        for table in self.d3.driver.find_elements(By.XPATH, xp.TABLES):
            try:
                head = table.find_element(By.XPATH, './thead')
                body = table.find_element(By.XPATH, './tbody')
                ed = ExtractData(body, head, meta)
                print(ed.headers)
                print(ed.body_data)

            except Exception as NoSuchElementException:
                continue

        # ExtractProduct(headers=head, data=body)


In [19]:
mc = MCMaster()

In [20]:
x = mc.get_family('https://www.mcmaster.com/products/pulling-and-lifting/', 'Pulling and Lifting')

2023-10-22 00:39:38.644 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/pulling-and-lifting/
2023-10-22 00:39:43.221 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/chain/


<undetected_chromedriver.webelement.WebElement (session="65cfb467b8bd43da7be7bea0262050a8", element="1988268AC7034A6524D42043534EC1FC_element_15")>
Chain https://www.mcmaster.com/products/chain/


2023-10-22 00:39:47.198 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/chain/grade-30-chain-not-for-lifting/


{'family_url': 'https://www.mcmaster.com/products/pulling-and-lifting/', 'family_name': 'Pulling and Lifting', 'category_url': 'https://www.mcmaster.com/products/chain/', 'category_name': 'Chain'} https://www.mcmaster.com/products/chain/grade-30-chain-not-for-lifting/


2023-10-22 00:39:52.762 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/chain/general-purpose-grade-80-chain-for-lifting/


...

2023-10-22 00:40:53.738 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/chain/chain-pull-cords/


{'title-2': ['Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', 'Carriers', '', ''], 'title-1': ['', '', '', '', '', 'Loop', 'Loop', '', '', '', '', '', '', '', 'Mounting\nBrackets', 'Mounting\nBrackets'], 'title-0': ['For Max.\nCable OD', 'For Total\nCable Wd.', 'Ht.', 'Wd.', 'Bend\nRadius', 'Ht.', 'Lg.,\nft.', 'Material', 'Color', 'Temperature\nRange, ° F', 'Mounting\nHardware Included', 'Length', ' ', 'Per Ft.', ' ', 'Pair']}
[['0.2"', '0.2"', '0.6"', '0.5"', '1.1"', '2.8"', '0.4', 'Nylon', 'Black', '-40° to 245°', 'No', '1 ft., 3 ft., 6 ft.', '4409T21', '$16.95', '4556T31', '$7.72'], ['0.3"', '0.3"', '0.6"', '0.7"', '1.1"', '2.8"', '0.4', 'Nylon', 'Black', '-40° to 245°', 'No', '1 ft., 3 ft., 6 ft.', '4409T22', '18.49', '4556T32', '8.67'], ['0.3"', '0.3"', '0.8"', '0.7"', '1.5"', '3.7"', '0.5', 'Nylon', 'Black', '-40° to 245°', 'No', '1 ft., 3 ft., 6 ft.', '4409T28', '21.53', '4

2023-10-22 00:40:57.847 | INFO     | __main__:get_cache:45 - Caching => https://www.mcmaster.com/products/chain/anchors-for-wheel-chocks/


{'title-1': ['Inside\nLink', 'Inside\nLink', '', '', 'Handle', 'Handle', 'Handle', 'Handle', 'Handle', 'Handle', '', '', '', '', ''], 'title-0': ['Lg.', 'Wd.', 'Sash Chain\nTrade Size', 'Cap., lbs.', 'Material', 'Dia.', 'Ht.', 'Wd.', 'Dp.', 'Color', ' ', '1 ft.', '2 ft.', '5 ft.', '8 ft.']}
[['0.3"', '0.15"', '8', 'Not Rated', 'Steel', '1"', '__', '__', '__', '__', '2678T1', '$20.53', '$21.50', '$24.45', '$27.37', 'Zinc-Plated Steel'], ['0.4"', '0.17"', '40', 'Not Rated', 'Steel', '1"', '__', '__', '__', '__', '2678T3', '21.57', '23.01', '27.27', '31.54', 'Zinc-Plated Steel'], ['0.26"', '0.12"', '8', 'Not Rated', 'Stainless Steel', '1"', '__', '__', '__', '__', '2678T6', '22.73', '27.52', '37.09', '46.64', '304 Stainless Steel'], ['0.34"', '0.15"', '35', 'Not Rated', 'Stainless Steel', '1"', '__', '__', '__', '__', '2678T7', '24.30', '27.43', '36.88', '46.31', '304 Stainless Steel'], ['1.11"', '0.21"', '__', '55', 'Plastic', '__', '1 5/8"', '3 1/8"', '3/4"', 'Black', '2679T1', '81.67',

KeyboardInterrupt: 

In [12]:
uc.Chrome(browser_executable_path='/Applications/Brave Browser.app/Contents/MacOS/Brave Browser', headless=False, use_subprocess=True)

<undetected_chromedriver.Chrome (session="b3265160bc818170aa257ed17d0047cf")>

In [8]:
uc.find_chrome_executable()

'/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'